Решить задачу регрессии.
Получить максимальную метрику качества (R2, RMSE).

Введение
Целью данной задачи является прогнозирование суммы пятничных распродаж в магазинах сети Walmart с помощью построения регрессионных моделей и их анализа.
Набор данных состоит из исторических данных, которые охватывают продажи с 2010-02-05 по 2012-11-01, предствленных в файле Walmart.csv
#Использовать приплайны.
#Выполнить поиск лучших параметров
#Кроссвалидация.
#Сохранить и загрузить моделль. 

In [11]:
import pandas as pd
import numpy as np
# from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from lightgbm import LGBMRegressor
# from sklearn.ensemble import RandomForestRegressor
# импорты всех нужных библиотек

In [12]:
df = pd.read_csv("Walmart.csv")
df.head() # загрузка датасета и вывод первых 5 строк

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
0,1,05-02-2010,1643690.90,0,42.31,2.572,211.096358,8.106
1,1,12-02-2010,1641957.44,1,38.51,2.548,211.242170,8.106
2,1,19-02-2010,1611968.17,0,39.93,2.514,211.289143,8.106
3,1,26-02-2010,1409727.59,0,46.63,2.561,211.319643,8.106
4,1,05-03-2010,1554806.68,0,46.50,2.625,211.350143,8.106


In [13]:
def get_day(row): # функция для получения дня из даты
    day = row.split('-')[0] # тут мы разбиваем дату на элементы (день, месяц и год)
    # и получаем первый элемент, - день
    if list(day)[0] == '0': # проверяем если число однозначное и если первое число 0...
        return int(list(day)[1]) # то мы просто вернем день без нуля спереди
    else:
        return int(day) # иначе возвращаем как оно есть

In [14]:
def get_month(row): # похожая функция
    month = row.split('-')[1] # тоже сплиттим и получаем месяц
    if list(month)[0] == '0': # тоже проверяем есть ли ноль спереди..
        return int(list(month)[1]) # .. и удаляем его
    else:
        return int(month) # иначе возвращаем как есть

In [15]:
df['Day'] = df['Date'].apply(get_day) # применение функций и создание новых столбцов
df['Month'] = df['Date'].apply(get_month)

In [16]:
X = df.drop(['Weekly_Sales', 'Date', 'Store'], axis=1)
y = df['Weekly_Sales']

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [18]:
pipe = Pipeline([('regressor', LGBMRegressor())])
grid = {
    'regressor__n_estimators': [100,200,300,400,500],
    'regressor__max_depth': [1,2,3,4,5]
}

In [19]:
gridsearch = GridSearchCV(pipe, grid, cv=5, scoring='r2')

In [20]:
gridsearch.fit(X_train,y_train)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000223 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1056
[LightGBM] [Info] Number of data points in the train set: 3603, number of used features: 7
[LightGBM] [Info] Start training from score 1041059.906662
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000184 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1054
[LightGBM] [Info] Number of data points in the train set: 3603, number of used features: 7
[LightGBM] [Info] Start training from score 1039561.844682
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000189 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1055
[LightGBM] [Info] Number of data points in the train set: 3603, number of used features: 7
[LightGBM] [Info] Sta

GridSearchCV(cv=5, estimator=Pipeline(steps=[('regressor', LGBMRegressor())]),
             param_grid={'regressor__max_depth': [1, 2, 3, 4, 5],
                         'regressor__n_estimators': [100, 200, 300, 400, 500]},
             scoring='r2')

In [21]:
y_pred = gridsearch.predict(X_test)
r2 = r2_score(y_test, y_pred)
rmse = (mean_squared_error(y_test, y_pred))**0.5

In [22]:
print(f"R2-score: {r2}")
print(f"RMSE: {rmse}") # вывод оценок модели

R2-score: 0.3361289283504686
RMSE: 460709.54558229627


In [36]:
import skl2onnx

In [40]:
onnx_model = skl2onnx.convert_sklearn(gridsearch.best_estimator_)

ValueError: Initial types are required. See usage of convert(...) in skl2onnx.convert for details